## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,-54.8000,-68.3000,49.66,76,75,17.27,light intensity drizzle
1,1,Mataura,NZ,-46.1927,168.8643,41.49,81,0,1.74,clear sky
2,2,Hermanus,ZA,-34.4187,19.2345,61.81,89,36,8.46,scattered clouds
3,3,Kidal,ML,18.4411,1.4078,80.62,14,98,10.07,overcast clouds
4,4,Nouadhibou,MR,20.9310,-17.0347,65.21,91,100,12.57,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Kidal,ML,18.4411,1.4078,80.62,14,98,10.07,overcast clouds
10,10,Kapaa,US,22.0752,-159.3190,78.78,84,75,17.27,broken clouds
12,12,Sao Joao Da Barra,BR,-21.6403,-41.0511,75.29,72,3,11.63,clear sky
14,14,Coahuayana,MX,18.7333,-103.6833,87.96,52,68,8.19,broken clouds
17,17,Rikitea,PF,-23.1203,-134.9692,78.42,69,1,12.66,clear sky
21,21,Hilo,US,19.7297,-155.0900,78.28,78,100,4.61,moderate rain
23,23,Salalah,OM,17.0151,54.0924,82.49,83,40,3.44,scattered clouds
24,24,Mersing,MY,2.4312,103.8405,78.67,86,51,2.73,heavy intensity rain
29,29,Mahebourg,MU,-20.4081,57.7000,76.24,87,43,14.23,light rain
32,32,Marsh Harbour,BS,26.5412,-77.0636,77.38,74,47,12.80,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        208
City           208
Country        208
Lat            208
Lng            208
Max Temp       208
Humidity       208
Cloudiness     208
Wind Speed     208
Description    208
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna() #no empty rows
preferred_cities_df.count()

City_ID        208
City           208
Country        208
Lat            208
Lng            208
Max Temp       208
Humidity       208
Cloudiness     208
Wind Speed     208
Description    208
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Kidal,ML,80.62,overcast clouds,18.4411,1.4078,
10,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
12,Sao Joao Da Barra,BR,75.29,clear sky,-21.6403,-41.0511,
14,Coahuayana,MX,87.96,broken clouds,18.7333,-103.6833,
17,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,
21,Hilo,US,78.28,moderate rain,19.7297,-155.0900,
23,Salalah,OM,82.49,scattered clouds,17.0151,54.0924,
24,Mersing,MY,78.67,heavy intensity rain,2.4312,103.8405,
29,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,
32,Marsh Harbour,BS,77.38,scattered clouds,26.5412,-77.0636,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.count()

City           208
Country        208
Max Temp       208
Description    208
Lat            208
Lng            208
Hotel Name     208
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name'] == '']
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count()

City           186
Country        186
Max Temp       186
Description    186
Lat            186
Lng            186
Hotel Name     186
dtype: int64

In [19]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
10,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Sao Joao Da Barra,BR,75.29,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
14,Coahuayana,MX,87.96,broken clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
17,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,People ThankYou
21,Hilo,US,78.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))